## Hypersonic Aircraft Model simulation

  Introduction to Robotics Course RBOT 101
  
  Brandeis Fall 2021/1 
  
  Author: Lekan Molu
  
  Date:   July 6, 2021

In [2]:
import control as ct
import numpy as np
import control.optimal as opt
import matplotlib.pyplot as plt

In [7]:
## Define system constants
# Source: https://arc.aiaa.org/doi/pdfplus/10.2514/2.4580?casa_token=es7MKh3DrHoAAAAA%3ADoflQuaXuyFyCnboBzXfaN_CrJx4ow0VCF4w7RcBdz4WOPn7IXUT3YiroytZCMrUHb1uiHBPfMUk&
m = 9375; 
S = 3603 #ft^3, ref area
rho = 0.0023769 # air density slug/ft3
CT = 0.0105 # thrust coefficient, approx of eq 18 in Stengel's paper
CL = 0.493 # lift coefficient eq A8 approximated
CD = 0.0082# drag coefficient
mu = 1.39*10**6 # graviatational constant, ft^3/s^2
I_yy = 7*10**6 # inertia momenta, slug-ft^2
V = 15060 # ft/sec
h = 110000 # ft
R_E = 20903500 # radius of the earth in ft
T = 4.6853*10**4 #lbf
cbar = 80 # ref length, ft
M = 15 # Mach number
alpha = 0.0315 # rad
alpha_0 = 0.0315 # trim value, rad
r = h + R_E # radial distance from the center of the earth

In [14]:
gamma = np.pi/4 # guess 45 degree is the flight path angle

A = [[ (1/m) * rho * S * (CT * np.cos(alpha) - CD) * V, -(mu/r**2) *np.cos(gamma), 0, (-1/2*m) * rho * S * CT * np.sin(alpha) * V**2, 0],
     [(1/2*m) * rho * S * CL - mu/(r**2 * V**2) + (1/r)*np.cos(gamma), -(V/r)*np.sin(gamma), 0, (1/2*m) *rho * S* CD *np.cos(alpha) * V**2, 0],
     [np.sin(gamma), V*np.cos(gamma), 0, 0, 0],
     [(-1/2*m) * rho * S * CL - (1/r)*np.cos(gamma) - mu/(r**2 * V**2), (V/r)*np.sin(gamma), 0,  (-1/2*m) * rho * S * CD * np.cos(alpha), 1],
     [0, 0, 0, 0,0]]

B = [[ 0, -1/m, 0], 
     [1/(m*V), 0, np.sin(alpha)/(m*V)],
     [0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]]

# For the simulation we need the full state output
sys = ct.ss2io(ct.ss(A, B, np.eye(5), 0,True)) # continuous time dynamics


In [15]:
sys

sys[0]

In [30]:
# simulate with nominal controls 
t = np.arange(0, 50, 1)
L = np.expand_dims(np.sin(2*np.pi*t), 1)
D = np.expand_dims(np.cos(2*np.pi*t), 1)
T = 2*L -D

In [32]:
ubar = np.concatenate((L, D, T), 1)
np.array(B).shape, ubar.shape

((5, 3), (50, 3))

In [ ]:
xdot = A@xbar + B@Ubar